This assignment explores two key concepts – sub-word modeling and convolutional networks – and applies them to the NMT system we built in the previous assignment. The Assignment 4 NMT model can be thought of as four stages:

1. Embedding layer: Converts raw input text (for both the source and target sentences) to a sequence of dense word vectors via lookup.
2. Encoder: A RNN that encodes the source sentence as a sequence of encoder hidden states.
3. Decoder: A RNN that operates over the target sentence and attends to the encoder hidden states to produce a sequence of decoder hidden states.
4. Output prediction layer: A linear layer with softmax that produces a probability distribution for the next target word on each decoder timestep.

- In Section 1 of this assignment, we will replace (1) with a character-based convolutional encoder
- and in Section 2 we will enhance (4) by adding a character-based LSTM decoder

# Section 1

![](../images/ex5_1.png)

## code for VocabEntry

In [4]:
from collections import Counter
from docopt import docopt
from itertools import chain
import json
import torch
from typing import List
from utils import read_corpus, pad_sents, pad_sents_char

[nltk_data] Downloading package punkt to /home/quantran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
class VocabEntry(object):
    """ Vocabulary Entry, i.e. structure containing either
    src or tgt language terms.
    """

    def __init__(self, word2id=None):
        """ Init VocabEntry Instance.
        @param word2id (dict): dictionary mapping words 2 indices
        """
        if word2id:
            self.word2id = word2id
        else:
            self.word2id = dict()
            self.word2id['<pad>'] = 0  # Pad Token
            self.word2id['<s>'] = 1  # Start Token
            self.word2id['</s>'] = 2  # End Token
            self.word2id['<unk>'] = 3  # Unknown Token
        self.unk_id = self.word2id['<unk>']
        self.id2word = {v: k for k, v in self.word2id.items()}

        ## Additions to the A4 code:
        self.char_list = list(
            """ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]""")

        self.char2id = dict()  # Converts characters to integers
        self.char2id['∏'] = 0  # <pad> token
        self.char2id['{'] = 1  # start of word token
        self.char2id['}'] = 2  # end of word token
        self.char2id['Û'] = 3  # <unk> token
        for i, c in enumerate(self.char_list):
            self.char2id[c] = len(self.char2id)
        self.char_pad = self.char2id['∏']
        self.char_unk = self.char2id['Û']
        self.start_of_word = self.char2id["{"]
        self.end_of_word = self.char2id["}"]
        assert self.start_of_word + 1 == self.end_of_word

        self.id2char = {v: k for k, v in self.char2id.items()}  # Converts integers to characters
        ## End additions to the A4 code

    def __getitem__(self, word):
        """ Retrieve word's index. Return the index for the unk
        token if the word is out of vocabulary.
        @param word (str): word to look up.
        @returns index (int): index of word
        """
        return self.word2id.get(word, self.unk_id)

    def __contains__(self, word):
        """ Check if word is captured by VocabEntry.
        @param word (str): word to look up
        @returns contains (bool): whether word is contained
        """
        return word in self.word2id

    def __setitem__(self, key, value):
        """ Raise error, if one tries to edit the VocabEntry.
        """
        raise ValueError('vocabulary is readonly')

    def __len__(self):
        """ Compute number of words in VocabEntry.
        @returns len (int): number of words in VocabEntry
        """
        return len(self.word2id)

    def __repr__(self):
        """ Representation of VocabEntry to be used
        when printing the object.
        """
        return 'Vocabulary[size=%d]' % len(self)

    def id2word(self, wid):
        """ Return mapping of index to word.
        @param wid (int): word index
        @returns word (str): word corresponding to index
        """
        return self.id2word[wid]

    def add(self, word):
        """ Add word to VocabEntry, if it is previously unseen.
        @param word (str): word to add to VocabEntry
        @return index (int): index that the word has been assigned
        """
        if word not in self:
            wid = self.word2id[word] = len(self)
            self.id2word[wid] = word
            return wid
        else:
            return self[word]

    def words2charindices(self, sents):
        """ Convert list of sentences of words into list of list of list of character indices.
        @param sents (list[list[str]]): sentence(s) in words
        @return word_ids (list[list[list[int]]]): sentence(s) in indices
        """
        return [[[self.char2id.get(c, self.char_unk) for c in ("{" + w + "}")] for w in s] for s in sents]

    def words2indices(self, sents):
        """ Convert list of sentences of words into list of list of indices.
        @param sents (list[list[str]]): sentence(s) in words
        @return word_ids (list[list[int]]): sentence(s) in indices
        """
        return [[self[w] for w in s] for s in sents]

    def indices2words(self, word_ids):
        """ Convert list of indices into words.
        @param word_ids (list[int]): list of word ids
        @return sents (list[str]): list of words
        """
        return [self.id2word[w_id] for w_id in word_ids]

    def to_input_tensor_char(self, sents: List[List[str]], device: torch.device) -> torch.Tensor:
        """ Convert list of sentences (words) into tensor with necessary padding for
        shorter sentences.

        @param sents (List[List[str]]): list of sentences (words)
        @param device: device on which to load the tensor, i.e. CPU or GPU

        @returns sents_var: tensor of (max_sentence_length, batch_size, max_word_length)
        """
        ### YOUR CODE HERE for part 1e
        ### TODO:
        ###     - Use `words2charindices()` from this file, which converts each character to its corresponding index in the
        ###       character-vocabulary.
        list_of_indices = self.words2charindices(sents) # list of list of list
        ###     - Use `pad_sents_char()` from utils.py, which pads all words to max_word_length of all words in the batch,
        ###       and pads all sentences to max length of all sentences in the batch. Read __init__ to see how to get
        ###       index of character-padding token
        sents_var = torch.Tensor(pad_sents_char(list_of_indices,self.char_pad)).permute(1,0,2)
        ###     - Connect these two parts to convert the resulting padded sentences to a torch tensor.
        return sents_var
        ### HINT:
        ###     - You may find .contiguous() useful after reshaping. Check the following links for more details:
        ###         https://pytorch.org/docs/stable/tensors.html#torch.Tensor.contiguous
        ###         https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view

        ### END YOUR CODE

    def to_input_tensor(self, sents: List[List[str]], device: torch.device) -> torch.Tensor:
        """ Convert list of sentences (words) into tensor with necessary padding for 
        shorter sentences.

        @param sents (List[List[str]]): list of sentences (words)
        @param device: device on which to load the tesnor, i.e. CPU or GPU

        @returns sents_var: tensor of (max_sentence_length, batch_size)
        """
        word_ids = self.words2indices(sents)
        sents_t = pad_sents(word_ids, self['<pad>'])
        sents_var = torch.tensor(sents_t, dtype=torch.long, device=device)
        return torch.t(sents_var)

    @staticmethod
    def from_corpus(corpus, size, freq_cutoff=2):
        """ Given a corpus construct a Vocab Entry.
        @param corpus (list[str]): corpus of text produced by read_corpus function
        @param size (int): # of words in vocabulary
        @param freq_cutoff (int): if word occurs n < freq_cutoff times, drop the word
        @returns vocab_entry (VocabEntry): VocabEntry instance produced from provided corpus
        """
        vocab_entry = VocabEntry()
        word_freq = Counter(chain(*corpus))
        valid_words = [w for w, v in word_freq.items() if v >= freq_cutoff]
        print('number of word types: {}, number of word types w/ frequency >= {}: {}'
              .format(len(word_freq), freq_cutoff, len(valid_words)))
        top_k_words = sorted(valid_words, key=lambda w: word_freq[w], reverse=True)[:size]
        for word in top_k_words:
            vocab_entry.add(word)
        return vocab_entry

In [6]:
temp = VocabEntry()
temp.words2charindices([['I','love','you'],['I','know']])

[[[1, 12, 2], [1, 41, 44, 51, 34, 2], [1, 54, 44, 50, 2]],
 [[1, 12, 2], [1, 40, 43, 44, 52, 2]]]

In [7]:
[temp.id2char[i] for i in [1, 40, 43, 44, 52, 2]]

['{', 'k', 'n', 'o', 'w', '}']

In [8]:
temp1 = torch.Tensor(temp.to_input_tensor_char([['I','loveee','you'],['I','know']],1))
temp1

tensor([[[ 1., 12.,  2.,  0.,  0.,  0.,  0.,  0.],
         [ 1., 12.,  2.,  0.,  0.,  0.,  0.,  0.]],

        [[ 1., 41., 44., 51., 34., 34., 34.,  2.],
         [ 1., 40., 43., 44., 52.,  2.,  0.,  0.]],

        [[ 1., 54., 44., 50.,  2.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]])

In [9]:
temp1.permute(1,0,2)

tensor([[[ 1., 12.,  2.,  0.,  0.,  0.,  0.,  0.],
         [ 1., 41., 44., 51., 34., 34., 34.,  2.],
         [ 1., 54., 44., 50.,  2.,  0.,  0.,  0.]],

        [[ 1., 12.,  2.,  0.,  0.,  0.,  0.,  0.],
         [ 1., 40., 43., 44., 52.,  2.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]])

In [10]:
temp.char2id['∏']

0

## code for highway

In [11]:
import torch.nn as nn
import torch.nn.functional as F
import torch

![](../images/ex5_2.png)

In [81]:
class Highway(nn.Module):
    def __init__(self,e_word):
        super().__init__()
        self.e_word = e_word
        self.w_proj = nn.Linear(e_word,e_word)
        self.w_gate = nn.Linear(e_word,e_word)
        # init linear weight and bias?
    def forward(self,x_conv_out):
        """
         
        raw_input: (max_sentence_length,bs,max_word_length aka m)
        which should be output of to_input_tensor_char()
        --char_emb()-->
        x_emb: (max_sentence_length,bs,max_word_length,e_char)
        with e_char is size of character embedding. 
        
        --reshape()-->
        x_reshaped: (max_sentence_length,bs,e_char,max_word_length)
        
        --cnn()-->
        x_conv: (max_sentence_length,bs,e_word,max_word_length-k+1)
        with k is kernel size,e_word is the desired word embedding size
        TODO: do a loop for each sentence?
        
        --relu_and_globalmaxpool()-->
        x_conv_out: (max_sentence_length,bs,e_word)
        
        --high_way()-->
        x_highway: (max_sentence_length,bs,e_word)
        
        --dropout()-->
        x_word_emb: (max_sentence_length,bs,e_word)
        
        input: x_conv_out shape (bs,max_sentence_length,e_word)
        output: x_highway shape (bs,max_sentence_length,e_word) (no dropout applied)
        """
        
        x_proj = F.relu(self.w_proj(x_conv_out))
        x_gate = torch.sigmoid(self.w_gate(x_conv_out))
        x_highway = x_gate * x_proj + (1-x_gate) * x_conv_out
        return x_highway

In [82]:
# test highway
temp_highway = Highway(2)
temp_conv_out = torch.randn(4,3,2)
temp_result = temp_highway(temp_conv_out)

In [83]:
temp_result.shape

torch.Size([4, 3, 2])

## code for cnn

In [ ]:
class CNN(nn.Module):
    def __init__(self,e_char,e_word,k):
        super().__init__()
        self.conv1d = nn.Conv1d(e_char,e_word,k)
    def forward(self,x_reshaped):
        """
        input: x_reshaped: (max_sentence_length,bs,e_char,max_word_length)
        
        output: x_conv: (max_sentence_length,bs,e_word,max_word_length-k+1)
        with k is kernel size,e_word is the desired word embedding size
        
        """

In [70]:
# test cnn
temp_conv = nn.Conv1d(3,4,2) #in_channels,out_channels,kernel_size

In [71]:
temp_w = temp_conv.weight.data
temp_b = temp_conv.bias.data

In [72]:
temp_x = torch.randn(1, 3, 2) # bs,in_channels aka emb size,number_of_items

In [73]:
temp3 = temp_conv(temp_x)
temp3

tensor([[[ 0.2538],
         [-0.0983],
         [ 0.3462],
         [ 0.8006]]], grad_fn=<SqueezeBackward1>)

In [74]:
temp3.shape # bs,out_channels,new_number_of_items

torch.Size([1, 4, 1])

In [75]:
(temp_w[0] * temp_x[0]).sum() + temp_b[0]

tensor(0.2538)